# Section 3.3 Investigating global trends of disease-specific co-expression networks at the edge level

In [1]:
import getpass
import json
import os
import sys
import time
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pyobo
import seaborn as sns
from network_utils import edge_file_path, create_network_from_edge_file, most_common, below_cutoff, \
    load_interactome, above_cutoff, remove_stats, percent_edges_in_other_edges, top_edges_subgraph #TODO change normal_subgraph
from tqdm import tqdm

In [2]:
getpass.getuser()

'rfigueiredo'

In [3]:
sys.version

'3.9.1 (v3.9.1:1e5d33e9b9, Dec  7 2020, 12:10:52) \n[Clang 6.0 (clang-600.0.57)]'

In [4]:
time.asctime()

'Fri Feb 26 23:49:48 2021'

## Load data
#### download data from https://doi.org/10.5281/zenodo.4572853 and place the folder 'data/' in the root of the repository, or if placed elsewhere, write the full path to 'data/' as the variable data_dir below

#### optional: select your desired location of the output figures

In [5]:
# replace here the location of the data dir if not structured as instructed
data_dir = "../data"

# optional, replace here the desired location of the output figures
figures_dir = os.path.join(os.path.expanduser("~"), "coexpath_figures")
os.makedirs(figures_dir, exist_ok=True)

##### Load co-expression networks

In [6]:
network_dict = {
    doid : create_network_from_edge_file(edge_file_path(os.path.join(data_dir, "final_coexprNets"), doid), doid)
    for doid in tqdm(os.listdir(os.path.join(data_dir, "final_coexprNets")), desc="Creating/loading network objects")
}

Creating/loading network objects: 100%|██████████| 64/64 [05:19<00:00,  4.99s/it]


##### Load cluster assignments

In [7]:
with open(os.path.join(data_dir, "misc_data", 'doid_group_clusters_dict.json'), 'r') as f:
    doids_per_cluster = json.load(f)

doid_cluster_mapping = {doid : cluster for cluster, doid_list in doids_per_cluster.items() if cluster != "Normal" for doid in doid_list}

##### Load disease name mappings

In [8]:
doid_name_mapping = pyobo.get_id_name_mapping('doid')
doid_id_mapping = pyobo.get_name_id_mapping('doid')

##### Load most common edges.
We did this on a separate machine as it is very computationally intensive, and saved the top 100,000 edges.
Alternatively, comment out this cell below and un-comment + run the one below it to perform this here

In [9]:
with open(os.path.join(data_dir, "misc_data", "100000most_common_edges.json"), 'r') as f:
    most_common_edges = json.load(f)

In [10]:
# most_common_edges = most_common(network_dict.values(), comparison="edges")

##### Load interactome network

In [11]:
interactome = load_interactome(os.path.join(data_dir, "interactome", "interactome_18_01_2021.tsv"))

## Edge stats

In [12]:
print("There are", len(above_cutoff(most_common_edges, 44)), "edges in at least 44 diseases")
print("There are", len(above_cutoff(most_common_edges, 32)), "edges in at least 32 diseases")

There are 21 edges in at least 44 diseases
There are 202 edges in at least 32 diseases


#### Among the most common edges in all diseases (top X)... what proportion can be found in the interactome? 

In [13]:
overlap_with_interactome = {}
percent_top_edges_in_interactome = {}
top_10_genes_in_overlap = {}
for i in tqdm(range(1000,10001,1000)):
    edges = remove_stats(most_common_edges[:i])
    overlap = []
    for edge in edges:
        if interactome.has_edge(*edge) or interactome.has_edge(*edge[::-1]):
            overlap.append(edge)
    overlap_with_interactome[i] = overlap
    percent_top_edges_in_interactome[i] = f"{len(overlap)/len(edges):.2%}"
    nodes = [node for edge in overlap for node in edge]
    counter_obj = Counter(nodes)
    top_10_genes_in_overlap[i] = counter_obj.most_common()[:10]

100%|██████████| 10/10 [00:00<00:00, 124.37it/s]


In [14]:
for k in percent_top_edges_in_interactome.keys():
    print("percent of top", k, "edges that are also in interactome:", 
          percent_top_edges_in_interactome[k])
    # print("\ttop 10 genes (gene, #connections):", top_10_genes_in_overlap[k]) # TODO uncomment if you want to display the top genes from the overlap

percent of top 1000 edges that are also in interactome: 4.00%
percent of top 2000 edges that are also in interactome: 3.20%
percent of top 3000 edges that are also in interactome: 2.90%
percent of top 4000 edges that are also in interactome: 2.53%
percent of top 5000 edges that are also in interactome: 2.32%
percent of top 6000 edges that are also in interactome: 2.08%
percent of top 7000 edges that are also in interactome: 1.94%
percent of top 8000 edges that are also in interactome: 1.85%
percent of top 9000 edges that are also in interactome: 1.70%
percent of top 10000 edges that are also in interactome: 1.66%


---
#### Among the most common edges in all diseases (top X)... what proportion can be found in the normal network?
#### ---> of those, what proportion can be found in the interactome?

In [15]:
sorted_normal_edges = sorted(network_dict["normal"].edges(data=True), key=lambda t: t[2].get('weight', 1), reverse = True)

In [16]:
overlap_with_normal = {}
percent_top_edges_in_normal = {}
top_10_genes_in_overlap = {}
for i in tqdm(range(1000,10001,1000)):
    edges = remove_stats(most_common_edges[:i])
    overlap = []
    for edge in edges:
        if normal_subgraph(i, sorted_normal_edges).has_edge(*edge):
            overlap.append(edge)
    overlap_with_normal[i] = overlap
    percent_top_edges_in_normal[i] = f"{len(overlap)/len(edges):.2%}"
    nodes = [node for edge in overlap for node in edge]
    counter_obj = Counter(nodes)
    top_10_genes_in_overlap[i] = counter_obj.most_common()[:10]

100%|██████████| 10/10 [3:24:06<00:00, 1224.67s/it]


In [17]:
for k in percent_top_edges_in_normal.keys():
    print(k,"edges: overlap with normal network:", percent_top_edges_in_normal[k], 
          f"overlap of those with interactome: {percent_edges_in_other_edges(overlap_with_normal[k],interactome.edges()):.2%}") 
    # print("\ttop 10 genes:", top_10_genes_in_overlap[k]) # TODO uncomment if you want to display the top genes from the overlap

1000 edges: overlap with normal network: 19.20% overlap of those with interactome: 7.81%
2000 edges: overlap with normal network: 19.85% overlap of those with interactome: 7.05%
3000 edges: overlap with normal network: 19.43% overlap of those with interactome: 5.66%
4000 edges: overlap with normal network: 18.93% overlap of those with interactome: 5.02%
5000 edges: overlap with normal network: 18.12% overlap of those with interactome: 4.64%
6000 edges: overlap with normal network: 17.65% overlap of those with interactome: 4.44%
7000 edges: overlap with normal network: 17.70% overlap of those with interactome: 4.28%
8000 edges: overlap with normal network: 16.94% overlap of those with interactome: 4.28%
9000 edges: overlap with normal network: 16.40% overlap of those with interactome: 4.07%
10000 edges: overlap with normal network: 16.48% overlap of those with interactome: 3.88%


---
#### Among the most common edges in all diseases that are not in the normal network (top X)... what proportion can be found in the interactome?

In [18]:
edges_only_in_diseases = [edge for edge, _ in most_common_edges if edge not in overlap_with_normal[10000]]

In [19]:
overlap_with_disease_edges = {}
percent_top_edges_in_diseases = {}
for i in tqdm(range(1000,10001,1000)):
    edges = edges_only_in_diseases[:i]
    overlap = []
    for edge in edges:
        if interactome.has_edge(*edge) or interactome.has_edge(*edge[::-1]):
            overlap.append(edge)
    overlap_with_disease_edges[i] = overlap
    percent_top_edges_in_diseases[i] = f"{len(overlap)/len(edges):.2%}"

100%|██████████| 10/10 [00:00<00:00, 64.81it/s]


In [20]:
for k in percent_top_edges_in_diseases.keys():
    print("percent of top", k, "disease-specific edges that are also in interactome:", percent_top_edges_in_diseases[k])

percent of top 1000 disease-specific edges that are also in interactome: 2.20%
percent of top 2000 disease-specific edges that are also in interactome: 2.30%
percent of top 3000 disease-specific edges that are also in interactome: 2.03%
percent of top 4000 disease-specific edges that are also in interactome: 1.73%
percent of top 5000 disease-specific edges that are also in interactome: 1.46%
percent of top 6000 disease-specific edges that are also in interactome: 1.45%
percent of top 7000 disease-specific edges that are also in interactome: 1.24%
percent of top 8000 disease-specific edges that are also in interactome: 1.20%
percent of top 9000 disease-specific edges that are also in interactome: 1.17%
percent of top 10000 disease-specific edges that are also in interactome: 1.05%


---
#### Interactome vs normal network

In [21]:
print(f"Overlap between interactome and equivalent proportion of top edges in normal network: \
        {percent_edges_in_other_edges(list(normal_subgraph(len(interactome.edges), sorted_normal_edges).edges()), list(interactome.edges())):.2%}" )

Overlap between interactome and equivalent proportion of top edges in normal network:         1.09%
